**DOWNLOAD THE PACS DATASET & SETUP THE ENVIRONMENT**

In [ ]:
# Download PACS Dataset Images
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!mv Homework3-PACS/PACS/ .
!rm -r Homework3-PACS/

# Download PACS Dataset Labels
!git clone https://github.com/silvia1993/DANN_Template/
!mv DANN_Template/txt_lists/art_painting.txt PACS/
!mv DANN_Template/txt_lists/cartoon.txt PACS/
!mv DANN_Template/txt_lists/photo.txt PACS/
!mv DANN_Template/txt_lists/sketch.txt PACS/
!rm -r DANN_Template/

# Install additional libraries
# %pip install torchmetrics

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032 (from 1)
Receiving objects: 100% (10032/10032), 174.13 MiB | 13.98 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23 (from 1)
Receiving objects: 100% (23/23), 33.86 KiB | 550.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 20.2 MB/s eta 0:00:00


**GLOBAL VARIABLES**

In [ ]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**DATASET MANAGEMENT**

In [ ]:
from torch.utils.data import Dataset

# Define the Dataset class
class PACSDataset(Dataset):
    def __init__(self):
        #...

    def __len__(self):
        #...

    def __getitem__(self, index):
        #...

IndentationError: expected an indented block after function definition on line 5 (<ipython-input-3-681fac41cdc7>, line 8)

**ARCHITECTURE SETUP**

In [ ]:
import torch.nn as nn

# Define AlexNet architecture class
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        # Category classifier
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
        # Domain classifier
        #...

    def forward(self, x):
        features = self.features(x)
        features = features.view(features.size(0), -1)
        class_outputs = self.classifier(features)
        #...
        return class_outputs

**OPTIMIZATION LOOP**

In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.models import AlexNet_Weights
import torch.nn.functional as F
from torchmetrics import Accuracy

#### DATA SETUP
# Define the transforms to use on images
dataset_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the Dataset object for training & testing
train_dataset = #...
test_dataset = #...

# Define the DataLoaders
train_loader = #...
test_loader = #...


#### ARCHITECTURE SETUP
# Create the Network Architecture object
model = AlexNet()
# Load pre-trained weights
model.load_state_dict(#...
# Overwrite the final classifier layer as we only have 7 classes in PACS
model.classifier[-1] = #...


#### TRAINING SETUP
# Move model to device before passing it to the optimizer
model = model.to(DEVICE)

# Create Optimizer & Scheduler objects
optimizer = torch.optim.SGD(#...
scheduler = torch.optim.lr_scheduler.StepLR(#...


#### TRAINING LOOP
#... for epoch in range(NUM_EPOCHS)...

#### TEST LOOP
#...

print(f'\nAccuracy on the target domain: {100 * accuracy:.2f}%')
